# 3.2.C CREATE TABLE with Foreign Keys

Table design principles using DDL commands, such as CREATE TABLE, DROP TABLE, ALTER TABLE, CREATE INDEX, DROP INDEX, CREATE VIEW, DROP VIEW.

Control columns values using constraints: primary keys, foreign keys, delete with CASCADE, CHECK, UNIQUE, NOT NULL, AUTOINCREMENT.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Foreign Key Constraints

A foreign key is a column or a set of columns in one table that refers to the primary key in another table. A foreign key establishes a relationship between two tables and ensures referential integrity. This means that the values in the foreign key column must match the values in the primary key column of the referenced table.

SQLite only supported foreign key constraints starting from version 3.6.19 (2009-10-14). By default, foreign key constraints are disabled in SQLite. To verify if your current SQLite version supports foreign key constraints, you can use the following command:

In [2]:
%%sql
PRAGMA foreign_keys;

 * sqlite:///
Done.


foreign_keys
0


---

The command returns an integer value: 

* 1: enable
* 0: disabled. 

If the command returns nothing, it means that your SQLite version doesn’t support foreign key constraints.

The foreign keys can be enabled or disabled using the following command:

```sql
PRAGMA foreign_keys = ON; -- enable
PRAGMA foreign_keys = OFF; -- disable
```

## Foreign Key Syntax

To create a foreign key constraint in SQLite, you can use the following syntax when creating a table:

```sql
CREATE TABLE table_name (
    column1 datatype,
    column2 datatype,
    ...
    FOREIGN KEY (column_name) REFERENCES other_table (other_column)
);
```

The table that contains the foreign key is called the child table, and the table that contains the primary key is called the parent table. The foreign key constraint ensures that any value in the child table's foreign key column must exist in the parent table's primary key column.

### Example

Let's create two tables: `students` and `courses`. The `students` table will have a foreign key that references the `courses` table.

In [3]:
%%sql

CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL
);

CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    student_name TEXT NOT NULL,
    course_id INTEGER,
    FOREIGN KEY (course_id) REFERENCES courses (course_id)
);

 * sqlite:///
Done.
Done.


[]

---

In this example, the `students` table has a foreign key `course_id` that references the `course_id` in the `courses` table. This means that any value in the `course_id` column of the `students` table must exist in the `course_id` column of the `courses` table.

If you try to insert a record into the `students` table with a `course_id` that does not exist in the `courses` table, SQLite will raise an error and prevent the insertion. This ensures referential integrity between the two tables.

Let's add some data to the `courses` table and then insert a record into the `students` table with a valid `course_id`. After that, we will try to insert a record with an invalid `course_id` to see how SQLite handles it.

In [4]:
%%sql

-- Turning foreign key constraints on
PRAGMA foreign_keys = ON;

INSERT INTO courses (course_id, course_name) VALUES
(1, 'Mathematics'),
(2, 'Science'),
(3, 'History');

INSERT INTO students (student_id, student_name, course_id) VALUES
(1, 'Alice', 1),
(2, 'Bob', 2),
(3, 'Charlie', 3),
(4, 'David', 1),
(5, 'Eve', 2);

 * sqlite:///
Done.
3 rows affected.
5 rows affected.


[]

In [5]:
%%sql
-- Error inserting a student with a non-existent course_id
INSERT INTO students (student_id, student_name, course_id) VALUES
(6, 'Frank', 4);  -- This will fail due to foreign key constraint

 * sqlite:///
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: -- Error inserting a student with a non-existent course_id
INSERT INTO students (student_id, student_name, course_id) VALUES
(6, 'Frank', 4);  -- This will fail due to foreign key constraint]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [6]:
%%sql
select * from students;

 * sqlite:///
Done.


student_id,student_name,course_id
1,Alice,1
2,Bob,2
3,Charlie,3
4,David,1
5,Eve,2


## Handling Foreign Key Violations

When you try to insert a record into the child table with a foreign key that does not exist in the parent table, SQLite will raise an error. But the bigger problem is when you delete or update a record in the parent table that is referenced by the child table. In this case, you have several options to handle foreign key violations:

1. **CASCADE**: If you delete or update a record in the parent table, all corresponding records in the child table will also be deleted or updated.
2. **SET NULL**: If you delete or update a record in the parent table, the corresponding foreign key values in the child table will be set to NULL.
3. **SET DEFAULT**: If you delete or update a record in the parent table, the corresponding foreign key values in the child table will be set to their default value.
4. **NO ACTION**: If you delete or update a record in the parent table, SQLite will raise an error and prevent the operation if there are corresponding records in the child table.
5. **RESTRICT**: Similar to NO ACTION, but it checks for foreign key violations immediately after the DELETE or UPDATE statement.

### CASCADE Example

The `CASCADE` action is the most common way to handle foreign key violations. It allows you to delete or update records in the parent table without worrying about the child table. When you delete or update a record in the parent table, all corresponding records in the child table will also be deleted or updated.

Let's create the `team` table and the `player` table with a foreign key constraint. The `player` table has a foreign key `team_id` that references the `team_id` in the `team` table. If we delete a record from the `team` table, all corresponding records in the `player` table will also be deleted.

In [7]:
%%sql
-- Create the team table
DROP TABLE IF EXISTS teams;

CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY,
    team_name TEXT NOT NULL
);

-- Insert some teams
INSERT INTO teams (team_id, team_name) VALUES
(1, 'Team A'),
(2, 'Team B'),
(3, 'Team C'),
(4, 'Team D'),
(5, 'Team E');

-- Create players table
DROP TABLE IF EXISTS players;

CREATE TABLE players (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT NOT NULL,
    team_id INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams (team_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE
);

-- Insert some players
INSERT INTO players (player_id, player_name, team_id) VALUES
(1, 'Player 1', 1),
(2, 'Player 2', 1),
(3, 'Player 3', 2),
(4, 'Player 4', 3),
(5, 'Player 5', 4),
(6, 'Player 6', 5);

SELECT t.team_id, team_name, player_id, player_name
FROM players p
    JOIN teams t
    ON p.team_id = t.team_id;

 * sqlite:///
Done.
Done.
5 rows affected.
Done.
Done.
6 rows affected.
Done.


team_id,team_name,player_id,player_name
1,Team A,1,Player 1
1,Team A,2,Player 2
2,Team B,3,Player 3
3,Team C,4,Player 4
4,Team D,5,Player 5
5,Team E,6,Player 6


Now let's delete a record from the `team` table and see how it affects the `player` table.

In [8]:
%%sql

DELETE FROM teams WHERE team_id = 1;

SELECT * FROM players;

 * sqlite:///
1 rows affected.
Done.


player_id,player_name,team_id
3,Player 3,2
4,Player 4,3
5,Player 5,4
6,Player 6,5


As you can see, players 1 and 2 are deleted from the `player` table when we delete the `team` with `team_id` 1. This is because of the `CASCADE` action we specified in the foreign key constraint.

### SET NULL Example

The `SET NULL` action allows you to delete or update records in the parent table without deleting or updating the corresponding records in the child table. Instead, the foreign key values in the child table will be set to NULL.

Let's create the `team` table and the `player` table with a foreign key constraint. The `player` table has a foreign key `team_id` that references the `team_id` in the `team` table. If we delete a record from the `team` table, the corresponding foreign key values in the `player` table will be set to NULL.

In [9]:
%%sql
DROP TABLE IF EXISTS team;
DROP TABLE IF EXISTS player;

CREATE TABLE team (
    team_id INTEGER PRIMARY KEY,
    team_name TEXT
);
CREATE TABLE player (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT,
    team_id INTEGER,
    FOREIGN KEY (team_id) REFERENCES team (team_id)   
        ON DELETE SET NULL
);

INSERT INTO team (team_id, team_name) VALUES (1, 'Team A'), (2, 'Team B');
INSERT INTO player (player_id, player_name, team_id) VALUES (1, 'Player 1', 1), (2, 'Player 2', 1), (3, 'Player 3', 2);

SELECT * FROM player;

 * sqlite:///
Done.
Done.
Done.
Done.
2 rows affected.
3 rows affected.
Done.


player_id,player_name,team_id
1,Player 1,1
2,Player 2,1
3,Player 3,2


In [10]:
%%sql 
DELETE FROM team WHERE team_id = 1;
SELECT * FROM player;

 * sqlite:///
1 rows affected.
Done.


player_id,player_name,team_id
1,Player 1,None
2,Player 2,None
3,Player 3,2


### SET DEFAULT Example

The `SET DEFAULT` action allows you to delete or update records in the parent table without deleting or updating the corresponding records in the child table. Instead, the foreign key values in the child table will be set to their default value.

Let's create the `team` table and the `player` table with a foreign key constraint. The `player` table has a foreign key `team_id` that references the `team_id` in the `team` table. If we delete a record from the `team` table, the corresponding foreign key values in the `player` table will be set to their default value.

In [11]:
%%sql

PRAGMA foreign_keys = OFF;
DROP TABLE IF EXISTS team;
DROP TABLE IF EXISTS player;
PRAGMA foreign_keys = ON;

CREATE TABLE team (
    team_id INTEGER PRIMARY KEY,
    team_name TEXT
);
CREATE TABLE player (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT,
    team_id INTEGER DEFAULT 2,
    FOREIGN KEY (team_id) REFERENCES team (team_id)
        ON DELETE SET DEFAULT
);

INSERT INTO team (team_id, team_name) VALUES (1, 'Team A'), (2, 'Team B');
INSERT INTO player (player_id, player_name, team_id) VALUES (1, 'Player 1', 1), (2, 'Player 2', 1), (3, 'Player 3', 2);
SELECT * FROM player;

 * sqlite:///
Done.
Done.
Done.
Done.
Done.
Done.
2 rows affected.
3 rows affected.
Done.


player_id,player_name,team_id
1,Player 1,1
2,Player 2,1
3,Player 3,2


In [12]:
%%sql 
DELETE FROM team WHERE team_id = 1;

SELECT * FROM player;

 * sqlite:///
1 rows affected.
Done.


player_id,player_name,team_id
1,Player 1,2
2,Player 2,2
3,Player 3,2


### RESTRICT and NO ACTION Example

The `RESTRICT` action is similar to the `NO ACTION` action. It prevents you from deleting or updating a record in the parent table if there are corresponding records in the child table. This means that if you try to delete or update a record in the parent table that is referenced by the child table, SQLite will raise an error and prevent the operation.

Let's create the `team` table and the `player` table with a foreign key constraint. The `player` table has a foreign key `team_id` that references the `team_id` in the `team` table. If we try to delete a record from the `team` table that is referenced by the `player` table, SQLite will raise an error and prevent the operation.


In [13]:
%%sql

PRAGMA foreign_keys = OFF;
DROP TABLE IF EXISTS team;
DROP TABLE IF EXISTS player;
PRAGMA foreign_keys = ON;

CREATE TABLE team (
    team_id INTEGER PRIMARY KEY,
    team_name TEXT NOT NULL
);
CREATE TABLE player (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT NOT NULL,
    team_id INTEGER,
    FOREIGN KEY (team_id) REFERENCES team (team_id) 
        ON DELETE RESTRICT
);
INSERT INTO team (team_id, team_name) VALUES (1, 'Team A');
INSERT INTO player (player_id, player_name, team_id) VALUES (1, 'Player 1', 1);
INSERT INTO player (player_id, player_name, team_id) VALUES (2, 'Player 2', 1);
DELETE FROM team WHERE team_id = 1; -- This will raise an error


 * sqlite:///
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: DELETE FROM team WHERE team_id = 1; -- This will raise an error]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


## Summary

In this lesson, we learned about foreign key constraints in SQLite. We learned how to create foreign key constraints using the `CREATE TABLE` statement and how to handle foreign key violations using different actions such as `CASCADE`, `SET NULL`, `SET DEFAULT`, `NO ACTION`, and `RESTRICT`. We also learned how to enable and disable foreign key constraints in SQLite.

## Exercises

1. Create a `departments` table with the following columns:
   - `department_id` (INTEGER, PRIMARY KEY)
   - `department_name` (TEXT, NOT NULL)
   - `location` (TEXT, NOT NULL)
2. Create an `employees` table with the following columns:
   - `employee_id` (INTEGER, PRIMARY KEY)
   - `first_name` (TEXT, NOT NULL)
   - `last_name` (TEXT, NOT NULL)
   - `department_id` (INTEGER, FOREIGN KEY REFERENCES departments(department_id) ON DELETE CASCADE)
3. Insert some sample data into the `departments` and `employees` tables.
4. Delete a department from the `departments` table and observe how it affects the `employees` table.
5. Change the foreign key constraint in the `employees` table to use `SET NULL` instead of `CASCADE`. Delete a department from the `departments` table and observe how it affects the `employees` table.
6. Change the foreign key constraint in the `employees` table to use `SET DEFAULT` instead of `CASCADE`. Delete a department from the `departments` table and observe how it affects the `employees` table.
7. Change the foreign key constraint in the `employees` table to use `RESTRICT` instead of `CASCADE`. Try to delete a department from the `departments` table that is referenced by the `employees` table and observe how it affects the operation.
8. Change the foreign key constraint in the `employees` table to use `NO ACTION` instead of `CASCADE`. Try to delete a department from the `departments` table that is referenced by the `employees` table and observe how it affects the operation.


In [14]:
%%sql
-- Your code in here

 * sqlite:///
Done.


[]

In [15]:
%%sql

-- More play area for you to test your SQL skills

 * sqlite:///
Done.


[]